In [3]:
# ? Could look at upserting cash flows and use them for the TD
#
from IPython.core.display_functions import display
from lusidtools.jupyter_tools import toggle_code

"""Term Deposit Valuation

Demonstrates pricing of a Term Deposit Investment.

Attributes
----------
instruments
valuation
lifecycle events
market data store
results store
quotes
"""

toggle_code("Hide docstring")

# Valuing and Rolling Term Deposits
In this notebook, we demonstrate how Term Deposits are handled in LUSID, as well as how to roll term deposits and perform valuations.

# Table of contents

In [4]:
# Import LUSID libraries
import lusid as lu
import lusid.models as lm
from lusidtools.lpt.lpt import to_date

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidjam.refreshing_token import RefreshingToken
import lusid.models as models

# Import Libraries
from datetime import datetime, timedelta
import pytz
import pandas as pd
import json
import os

# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.4f}'.format)

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lu.utilities.ApiClientFactory(
        token=RefreshingToken(),
        api_secrets_filename = secrets_path,
        app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.11167.0


In [5]:
# Initiate the LUSID APIs required for the notebook
instruments_api = api_factory.build(lu.api.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lu.AggregationApi)
quotes_api = api_factory.build(lu.QuotesApi)

In [6]:
# Define scopes
scope = "ibor"
portfolio_code = "TermDepositExamplePortfolio"

# 1. Create Portfolio
We must first create portfolio to keep the term depoists in.

In [7]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'TermDepositExamplePortfolio' because it already exists in scope 'ibor'.


# 2. Create Term Deposit Instruments
Next we must create an instrument to represent the Term Deposit. In LUSID the rate of the term deposit must be defined at creation and cannot be adjusted.

In [8]:
# Not sure what this is?
flow_conventions = lm.FlowConventions(
    currency="USD",
    payment_frequency="6M",
    roll_convention="F",
    day_count_convention="Actual360",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0,
)

In [9]:
# Define key dates for the term deposits in this example
start_date = datetime(2020, 3, 18, 00, tzinfo=pytz.utc)
maturity_date = datetime(2020, 9, 18, 00, tzinfo=pytz.utc)

In [15]:
# Helper method to create term deposit instrument
def create_td_instrument(
    name,
    identifier,
    start_date,
    maturity_date,
    contract_size,
    flow_convention,
    rate,
    dom_ccy
):
    td_instrument = lm.TermDeposit(
        start_date=start_date,
        maturity_date=maturity_date,
        contract_size=contract_size,
        flow_convention=flow_convention,
        rate=rate,
        dom_ccy=dom_ccy,
        instrument_type="TermDeposit",
        local_vars_configuration=None)

    td_definition = lm.InstrumentDefinition(
        name=name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(identifier)},
        definition=td_instrument
    )

    upsert_request = {identifier: td_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    td_luid = upsert_response.values[identifier]
    print(f"Created instrument with LUID:{td_luid.lusid_instrument_id} and ClientInternal:{identifier}")
    return td_luid.lusid_instrument_id

In [16]:
td_luid_a = create_td_instrument(
    "TD Example Inst A",
    "TD_Inst_A",
    start_date,
    maturity_date,
    1000,
    flow_conventions,
    0.03,
    "USD"
)

td_luid_a

Created instrument with LUID:LUID_00003DET and ClientInternal:TD_Inst_A


'LUID_00003DET'

# 3. Add Transactions
Now we will upload the initial StockIn transactions, adding the Term Deposit instruments to the portfolio

In [17]:
# Load transaction from csv
transactions_df = pd.read_csv("data/term_deposit_transactions.csv")
transactions_df.head()

,txn_id,type,client_id,trade_date,settlement_date,quantity,price,total_consideration,currency
0,txn001,StockIn,TD_Inst_A,2020-03-18T00:00:00Z,2020-03-20T00:00:00Z,"1,000,000",1,"1,000,000.00",USD


In [18]:
# Load transactions to LUSID}
transaction_request = [
    lm.TransactionRequest(
        transaction_id=txn["txn_id"],
        type=txn["type"],
        instrument_identifiers={
            "Instrument/default/ClientInternal": txn["client_id"]
        },
        transaction_date=to_date(txn["trade_date"]),
        settlement_date=to_date(txn["settlement_date"]).isoformat(),
        units=txn["quantity"],
        transaction_price=lm.TransactionPrice(price=txn["price"], type="Price"),
        total_consideration=lm.CurrencyAndAmount(
            amount=txn["total_consideration"], currency=txn["currency"]
        ),
    )
    for index, txn in transactions_df.iterrows()
]


response = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=transaction_request
)

print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2023-04-21 12:00:19.987647+00:00


In [19]:
# Helper method to view the transaction in LUSID up to a specific date
def get_transactions_todate(to_transaction_date):
    get_transactions_reponse = transaction_portfolios_api.get_transactions(
        scope = scope,
        code = portfolio_code,
        to_transaction_date=to_transaction_date
    )
    df = lusid_response_to_data_frame(get_transactions_reponse)
    return df[['transaction_id','type','instrument_identifiers.Instrument/default/ClientInternal','instrument_uid','transaction_date','settlement_date','transaction_price.price','total_consideration.amount','units']]

# 4. Get Holdings

In [20]:
# Helper method to view holdings in LUSID
def get_holdings(date):
    resp = transaction_portfolios_api.get_holdings(scope=scope,
                                         code=portfolio_code,
                                         property_keys=["Instrument/default/Name"],
                                        effective_at=date)
    df = lusid_response_to_data_frame(resp)
    return df[["instrument_scope","instrument_uid","properties.Instrument/default/Name.value.label_value","units","cost.amount","holding_type_name"]]

In [21]:
get_holdings(start_date)

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,LUID_00003DET,TD Example Inst A,"1,000,000.0000","1,000,000.0000",Position


# 5. Running a Valuation
In this section we will setup and run a valuation on our portfolio

## 5.1 Setup recipe
To run a valuation in LUSID we must first define a recipe.
The helper method below defines a recipe for a Term Deposit instrument by passing in `instrument_type="TermDeposit"` to the `VendorModelRule`.

In [22]:
# Define a method for creating the recipe
def create_recipe(recipe_code, scope, model):
    # Populate recipe parameters
    recipe = lm.ConfigurationRecipe(
        scope=scope,
        code=recipe_code,
        market=lm.MarketContext(
            market_rules=[
                lm.MarketDataKeyRule(
                    key="Equity.ClientInternal.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field="mid",
                    quote_interval="5D.0D"
                ),
                lm.MarketDataKeyRule(
                    key="FxForwards.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="1Y.0D"
                )
            ],
            options=lm.MarketOptions(
                attempt_to_infer_missing_fx=True,
                default_scope=scope
            )
        ),
        # Set the valuation model - curve with no discounting
        pricing=lm.PricingContext(
            model_rules=[
                lm.VendorModelRule(
                    supplier="Lusid",
                    model_name=model,
                    instrument_type="TermDeposit",
                    parameters="{}",
                )
            ],
            options=lm.PricingOptions(
                produce_separate_result_for_linear_otc_legs=False
            )
        ),
    )

    response = configuration_recipe_api.upsert_configuration_recipe(
                upsert_recipe_request=lm.UpsertRecipeRequest(
                    configuration_recipe=recipe
                )
            )

    return response

LUSID supports SimpleStatic, Discounting and ConstantTimeValueOfMoney pricing models for term deposits. In this notebook we will look at SimpleStatic and ConstantTimeValueOfMoney models.

In [23]:
# Define recipe names
CTVoMRecipe = "TermDepositRecipeCTVoM"
SSRecipe = "TermDepositRecipeSS"

# Create recipes in LUSID
create_recipe(CTVoMRecipe, scope, "ConstantTimeValueOfMoney")
create_recipe(SSRecipe, scope, "SimpleStatic")

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2A6F8P2DJ:0000017B',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2023, 4, 21, 19, 19, 21, 487247, tzinfo=tzutc())}

In [24]:
# Helper method runs a valuation on the portfolio for a given date and recipe
def perform_valuation(recipe, date):
    # Create valuation request
    valuation_request = lm.ValuationRequest(
        # Choose recipe to use
        recipe_id = lm.ResourceId(scope = scope, code = recipe),
        group_by = ['Instrument/default/LusidInstrumentId'],
        metrics = [
        {"key": "Instrument/default/LusidInstrumentId", "op": "Value"},
        {"key": "Instrument/default/Name", "op": "Value"},              # Reports the friendly name of the underlying instrument
        {"key": "Analytic/default/ValuationDate", "op": "Value"},       # Confirms the valuation date
        {"key": "Valuation/PV", "op": "Sum"}],                          # Calculates cost in GBP (the portfolio currency)
        # Identify portfolio to value
        portfolio_entity_ids = [lm.PortfolioEntityId(scope = scope, code = portfolio_code)],
        # Make date of valuation conditional on effective at date passed into function
        valuation_schedule = lm.ValuationSchedule(effective_at=date),
    )

    # Get portfolio valuation
    val_data = aggregation_api.get_valuation(valuation_request = valuation_request).data

    # Turn valuation response into pandas dataframe
    vals_df = pd.DataFrame(val_data)
    try:
        return vals_df.drop('Aggregation Errors', axis=1)
    except:
        return vals_df

In [25]:
quote_request = {
   "quote_request" : lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=td_luid_a,
                instrument_id_type="LusidInstrumentId",
                quote_type="Price",
                field="mid",
            ),
            effective_at="2010-01-01T18:00:00Z",
        ),
        metric_value=lm.MetricValue(value=0.05, unit="USD"),
    )}


# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=quote_request)
print(response)

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2HO083LKH:000000B7',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'quote_request': {'as_at': datetime.datetime(2023, 4, 21, 19, 19, 21, 769048, tzinfo=tzutc()),
                              'cut_label': '',
                              'lineage': '',
                              'metric_value': {'unit': 'USD', 'value': 1.05},
                              'quote_id': {'effective_at': '2010-01-01T18:00:00.0000000+00:00',
                                           'quote_series_id': {'field': 'mid',
                                                               'instrument_id': 'LUID_00003DET',
                                                               'instrument_id_type': 'LusidInstrumentId',
                  

In [26]:
perform_valuation(CTVoMRecipe,start_date)

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,LUID_00003DET,TD Example Inst A,2020-03-18T00:00:00.0000000+00:00,"1,015,333,333.3333"


In [27]:
# ?? This doesn't work
# perform_valuation(SSRecipe,start_date)

In [28]:
# Helper method - used in the scenarios below - gives a valuation of a specific instrument at a given date
def get_instr_valuation(recipe, date, luid):
    df = pd.DataFrame(perform_valuation(recipe, date))
    instr_val = df[df["Instrument/default/LusidInstrumentId"]==luid]
    return instr_val.iloc[0][3]


## 6. Rolling a Term Deposit
For these scenarios we will always use the Constant Time Value of Money pricing model.
In LUSID we handle rolling a term deposit by selling the term deposit before maturity and buying it back again.
This section will show 2 example scenarios:
1. Rolling with a decrease to principal market value
2. Rolling with an increase to interest


### 6.1 Rolling with a decrease to principal market value

#### 6.1.1 Sell Term Deposit just before maturity
To roll the deposit we first sell the deposit before maturity. We use the instrument valuation calculated on the sale transaction date for the sale price.

In [29]:
# For rolling term deposits we will define new start dates and maturity dates.
roll1_start_date = maturity_date
roll1_maturity_date = datetime(2021,3,19,00,tzinfo=pytz.utc)

pre_maturity_sale_date = maturity_date - timedelta(2)

In [40]:
def get_cashflow(start_date, maturity_date, recipe):
    resp = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
        scope=scope,
        code=portfolio_code,
        effective_at=maturity_date- timedelta(days=1),
        window_start=start_date,
        window_end=maturity_date + timedelta(days=2),
        recipe_id_scope=scope,
        recipe_id_code=recipe
    )

    print(lusid_response_to_data_frame(resp))
    # [['transaction_id','type','instrument_identifiers.Instrument/default/LusidInstrumentId','units','total_consideration.amount']])

    return resp.values

txn = get_cashflow(start_date, maturity_date, CTVoMRecipe)

                                   transaction_id      type  \
0  txn001-LUID_00003DET-20200918-Cash-USD-Receive  CashFlow   

  instrument_identifiers.Instrument/default/LusidInstrumentId  \
0                                      LUID_00003DET            

  instrument_scope instrument_uid          transaction_date  \
0          default  LUID_00003DET 2020-09-18 00:00:00+00:00   

            settlement_date              units  transaction_price.price  \
0 2020-09-18 00:00:00+00:00 1,015,333,333.3333                   1.0000   

  transaction_price.type  ...  total_consideration.currency exchange_rate  \
0                  Price  ...                           USD        1.0000   

   transaction_currency properties.Transaction/default/ParentLuid.key  \
0                   USD                Transaction/default/ParentLuid   

  properties.Transaction/default/ParentLuid.value.label_value  \
0                                      LUID_00003DET            

  properties.Transaction/defaul

In [41]:
transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=txn
)

{'href': 'https://liberty.lusid.com/api/api/transactionportfolios/ibor/TermDepositExamplePortfolio/transactions?asAt=2023-04-21T19%3A33%3A17.2847990%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://liberty.lusid.com/api/api/portfolios/ibor/TermDepositExamplePortfolio?effectiveAt=2020-09-18T00%3A00%3A00.0000000%2B00%3A00&asAt=2023-04-21T19%3A33%3A17.2847990%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://liberty.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://liberty.lusid.com/app/insights/logs/0HMQ2A5QDS178:0000006E',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {},
 'version': {'a

In [44]:
get_transactions_todate(pre_maturity_sale_date+timedelta(3))

,transaction_id,type,instrument_identifiers.Instrument/default/ClientInternal,instrument_uid,transaction_date,settlement_date,transaction_price.price,total_consideration.amount,units
0,txn001,StockIn,TD_Inst_A,LUID_00003DET,2020-03-18 00:00:00+00:00,2020-03-20 00:00:00+00:00,1.0000,"1,000,000.0000","1,000,000.0000"
1,txn001-LUID_00003DET-20200918-Cash-USD-Receive,CashFlow,NaN,LUID_00003DET,2020-09-18 00:00:00+00:00,2020-09-18 00:00:00+00:00,1.0000,"1,015,333,333.3333","1,015,333,333.3333"


When we look at the holdings below we can see the Term Deposit has been replaced with cash in our portfolio

In [46]:
get_holdings(maturity_date+timedelta(3))

,instrument_scope,instrument_uid,properties.Instrument/default/Name.value.label_value,units,cost.amount,holding_type_name
0,default,LUID_00003DET,TD Example Inst A,"1,000,000.0000","1,000,000.0000",Position
1,default,CCY_USD,USD,"1,015,333,333.3333","1,015,333,333.3300",Balance


In [48]:
perform_valuation(CTVoMRecipe, maturity_date)

,Instrument/default/LusidInstrumentId,Instrument/default/Name,Analytic/default/ValuationDate,Sum(Valuation/PV)
0,LUID_00003DET,TD Example Inst A,2020-09-18T00:00:00.0000000+00:00,"1,015,333,333.3333"
1,CCY_USD,USD,2020-09-18T00:00:00.0000000+00:00,"1,015,333,333.3333"


### 6.1.2 Buy term deposit back with a decrease to market value
In the cell below we will define a new Term Deposit B Instrument which has a start date at the end of the first Term Deposit A and a maturity date 6 months later. In all other aspects it is the same as the first term deposit instrument which was sold above.


In [35]:
td_luid_b  = create_td_instrument(
    "TD Example Inst B",
    "TD_Inst_B",
    roll1_start_date,
    roll1_maturity_date,
    1000,
    flow_conventions,
    0.03,
    "USD"
)

print(f"Created instrument with LUID: {td_luid_b}")

Created instrument with LUID:LUID_00003DEU and ClientInternal:TD_Inst_B
Created instrument with LUID: LUID_00003DEU


We will now buy the new Term Deposit B at a price lower than the value it was sold at in the previous transaction.

In [36]:
buy_back_price = pre_maturity_valuation - 1000
buy_back_date = pre_maturity_sale_date+timedelta(1)

buy_td = lm.TransactionRequest(
    transaction_id="txn003",
    type="Buy",
    instrument_identifiers={"Instrument/default/LusidInstrumentId": td_luid_b},
    transaction_date=buy_back_date.isoformat(),
    settlement_date=(buy_back_date+timedelta(3)).isoformat(),
    units=1000000,
    transaction_price=lm.TransactionPrice(price=1,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=buy_back_price,currency="USD"),
    exchange_rate=1,
    transaction_currency="USD"
)

transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=[buy_td]
)

NameError: name 'pre_maturity_valuation' is not defined

In [ ]:
get_transactions_todate(buy_back_date + timedelta(days=4))

In [ ]:
get_holdings(buy_back_date)

In [ ]:
# ?? Whats going on with the dates here
perform_valuation(CTVoMRecipe, buy_back_date+timedelta(3))

## 6.2 Rolling with an increase to interest
### 6.2.1 Sell Term Deposit just before maturity
As in the previous rolling example, we first sell the deposit before maturity. We use the instrument valuation calculated on the sale transaction date for the sale price.

In [ ]:
# For rolling term deposits we will define new start dates and maturity dates.
roll2_start_date = roll1_maturity_date - timedelta(1)
roll2_maturity_date = roll2_start_date + timedelta(7)

In [ ]:
pre_maturity_sale_date = roll1_maturity_date - timedelta(days=2)
pre_maturity_valuation = get_instr_valuation(CTVoMRecipe, pre_maturity_sale_date, td_luid_b)

sell_td = lm.TransactionRequest(
    transaction_id="txn004",
    type="Sell",
    instrument_identifiers={"Instrument/default/LusidInstrumentId": td_luid_b},
    transaction_date=  pre_maturity_sale_date.isoformat(),
    settlement_date=(pre_maturity_sale_date+timedelta(3)).isoformat(),
    units=1000000,
    transaction_price=lm.TransactionPrice(price=1,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=pre_maturity_valuation,currency="USD")
)

transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=[sell_td]
)

In [ ]:
get_transactions_todate(pre_maturity_sale_date)

In [ ]:
get_holdings(pre_maturity_sale_date)

In [ ]:
perform_valuation(CTVoMRecipe, pre_maturity_sale_date)

### 6.2.2 Buy term deposit back with an Increased Rate
We will create a new instrument below with a higher rate of 0.05 -> 5% (the original Term Deposit had a rate of 3%).

In [ ]:
# Define a new instrument with a higher rate
td_luid_c = create_td_instrument(
    "TD Example Instrument C",
    "TD_EXAMPLE_C",
    roll2_start_date,
    roll2_maturity_date,
    1000,
    flow_conventions,
    0.05,
    "GBP"
)

This new instrument will be purchased at the same price the previous term deposit was sold at.

In [ ]:
buy_back_date = pre_maturity_sale_date+timedelta(1)

buy_td = lm.TransactionRequest(
    transaction_id="txn005",
    type="Buy",
    instrument_identifiers={"Instrument/default/LusidInstrumentId": td_luid_c},
    transaction_date=buy_back_date.isoformat(),
    settlement_date=(buy_back_date+timedelta(3)).isoformat(),
    units=1000000,
    transaction_price=lm.TransactionPrice(price=1,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=pre_maturity_valuation,currency="USD"),
    exchange_rate=1,
    transaction_currency="USD"
)

transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=[buy_td]
)

In [ ]:
get_transactions_todate(buy_back_date + timedelta(days=4))

The holdings shown below reflect the $1,000 price difference in the sell price of Term Deposit A and the buy price of Term Deposit B, in the USD cash holding.

In [ ]:
get_holdings(buy_back_date)

In [ ]:
perform_valuation(CTVoMRecipe, buy_back_date+timedelta(3))

### 5.2 Valuations using CTVoM

In [ ]:
# Before first roll
perform_valuation(CTVoMRecipe, datetime(2034,12,1,00, tzinfo=pytz.utc))

In [ ]:
# After first roll
perform_valuation(CTVoMRecipe, datetime(2034,12,14,00, tzinfo=pytz.utc))

In [ ]:
# After second roll
perform_valuation(CTVoMRecipe, portfolio_code,  datetime(2034,12,24,00, tzinfo=pytz.utc))

In [ ]:
# After maturity
perform_valuation(CTVoMRecipe, portfolio_code,  datetime(2035,1,1,00, tzinfo=pytz.utc))

### 5.3 Valuations using SS

In [ ]:
# Before first roll
perform_valuation(SSRecipe, portfolio_code,  datetime(2034,12,1,00, tzinfo=pytz.utc))

In [ ]:
# After first roll
perform_valuation(SSRecipe, portfolio_code,  datetime(2034,12,14,00, tzinfo=pytz.utc))

In [ ]:
# After second roll
perform_valuation(SSRecipe, portfolio_code,  datetime(2034,12,24,00, tzinfo=pytz.utc))

In [ ]:
# After maturity
perform_valuation(CTVoMRecipe, portfolio_code,  datetime(2035,1,1,00, tzinfo=pytz.utc))

? How do I feed valuation price into sale price
? Should valuation prices change with time
? Does the second purchase after the roll need to have a later maturity

In [ ]:
# Book a sell transaction against the term deposit
preMaturityValuation = instA_valuation(rolling_date)

sell_td = lm.TransactionRequest(
    transaction_id="txn003",
    type="Sell",
    instrument_identifiers={"Instrument/default/ClientInternal": "TD_EXAMPLE_A"},
    transaction_date=rolling_date.isoformat(),
    settlement_date=(rolling_date + timedelta(days=30)).isoformat(),
    units=75,
    transaction_price=lm.TransactionPrice(price=0,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=preMaturityValuation,currency="USD"),
    exchange_rate=1,
    transaction_currency="USD"
)

response = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                    code=portfolio_code,
                                                    transaction_request=[sell_td])
response

In [ ]:
# # Buy TD back with lower market value
#
# sell_td = lm.TransactionRequest(
#     transaction_id="txn004",
#     type="Sell",
#     instrument_identifiers={"Instrument/default/ClientInternal": "TD_EXAMPLE_A"},
#     transaction_date=rolling_date.isoformat(),
#     settlement_date=(rolling_date + timedelta(days=30)).isoformat(),
#     units=75,
#     transaction_price=lm.TransactionPrice(price=0,type="Price"),
#     total_consideration=lm.CurrencyAndAmount(amount=preMaturityValuation,currency="USD"),
#     exchange_rate=1,
#     transaction_currency="USD"
# )